## Imports

In [1]:
# Imports 
import torch

## Constructing a model

In [11]:
# Define a Model
class Net(torch.nn.Module):
    """ Constructor """
    def __init__(self):
        super(Net, self).__init__()
        
        self.stack = torch.nn.Sequential(
            torch.nn.Linear(128, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1),
            torch.nn.ReLU()
        )
        
    """
        Define forward-pass
    """
    def forward(self, x):
        return self.stack(x)

# Print Model    
print(model)

Net(
  (stack): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): ReLU()
  )
)


In [12]:
# Check if CUDA drivers are available
device_type = "cuda" if torch.cuda.is_available() else "cpu"
model = Net().to(device_type)

## Training a model

In [14]:
# Define a loss function
loss = torch.nn.MSELoss # Mean squared error

# Define an optimizer
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # Stochastic Gradient Descent

## Sources:
The 

**[PyTorch Quickstart Tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)**
